## Import Packages

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
#image processing
from PIL import ImageFilter, Image

In [2]:
img = Image.open("test.jpg")
img.filter(ImageFilter.SHARPEN).save('test_1.jpg', 'JPEG', quality=1000) 
del img

##### Image Processing

In [3]:
#load color image
original_image = cv2.imread('test_1.jpg')
image_color = original_image.copy()

In [4]:
#load image
img = cv2.imread('test_1.jpg',0)
# Thresholding the image
(thresh, img_bin) = cv2.threshold(img, 128, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# Invert the image
img_bin = 255-img_bin 
cv2.imwrite("binary.jpg",img_bin)

True

In [5]:
def show_image(image):
    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
#get all pixeles coordinates in a contour
def get_cord(x,y,w,h):
    idx=0
    coords = []
    for i in range (x,x+w):
        for j in range (y,y+h):
            coords.insert(idx,[i,j])
            #print(idx, coords)
            idx+=1
            
    return coords

In [7]:
def adjust_gamma(image, gamma=2.8):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

##### Create Kernel

define two kernels
-  Kernel to detect horizontal lines. 
-  Kernel to detect vertical lines.

In [8]:
# Defining a kernel length
kernel_length = np.array(img).shape[1]//165

# A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))

# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))

# A kernel of (3 X 3) ones.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

In [9]:
def find_boxes(img_bin):
    # Defining a kernel length
    kernel_length = np.array(img).shape[1]//195

    # A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
    verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))

    # A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))

    # A kernel of (3 X 3) ones.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    
    #### Morphological operation to detect vertical lines from an image
    img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
    verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
    cv2.imwrite("verticle_lines.jpg",verticle_lines_img)
    
    # Morphological operation to detect horizontal lines from an image
    
    img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
    cv2.imwrite("horizontal_lines.jpg",horizontal_lines_img)
    # Weighting parameters, this will decide the quantity of an image to be added to make a new image.
    alpha = 0.5
    beta = 1.0 - alpha

    # This function helps to add two image with specific weight parameter to get a third image as summation of two image.
    img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
    #img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=1)
    img_final_bin = ~img_final_bin
    return cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

In [10]:
(thres, img_final_bin) = find_boxes(img_bin)
cv2.imwrite("img_final_bin.jpg",img_final_bin)

True

In [11]:
def sort_contours(cnts, method="top-to-bottom"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
 
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
 
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
 
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
 
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [12]:
# Find contours for image, which will detect all the boxes
contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Sort all the contours by top to bottom.
(contours, boundingBoxes) = sort_contours(contours, method="top-to-bottom")

### Identifying Contours

In [13]:
idx = 0
cropped_path = 'Cropped/'
for c in contours:
    # Returns the location and width,height for every contour
    x, y, w, h = cv2.boundingRect(c)
    #get all the pixels coordinates under contour
    coordinates=[]
    crd = get_cord(x,y,w,h)
    coordinates.append(crd)
# If the box height is greater then 20, widht is >80, then only save it as a box in "cropped/" folder.
    if (w > 12 and h > 15) and w > 3*h and w<420 and h<40:
        idx += 1
        new_img = img[y:y+h, x:x+w]
        cv2.imwrite(cropped_path+str(idx) + '.png', new_img)
        #original_image[y:y+h, x:x+w] = adjust_gamma(original_image[y:y+h, x:x+w]) #Highlight Contours

        #draw contours
        img_outlined=cv2.rectangle(image_color,(x,y),(x+w,y+h),(0,0,255),1)
show_image(img_outlined)
del img_outlined

### Random testing and Image Processing

In [14]:
img = cv2.imread('test_1.jpg',0)
edges = cv2.Canny(img,100,200,L2gradient=True)
#cv2.imshow('Edges',edges)
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
cv2.imwrite('Opening.png', opening)
cv2.imshow('applied opening', opening)
#dilation = cv2.dilate(edges,kernel,iterations = 1)
#cv2.imshow('dilated_image',dilation)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Following function matches the right click coordinates with the pixel-values of each contour

In [15]:
drawing = False # true if mouse is pressed
mode = True # if True, draw rectangle.
def mouse_func(event, x, y, flags, param):
    global drawing,mode
    # 2 is the value for the right mouse click
    if event == 2:
        drawing = True
        cord = [x,y] #right click co-ordinates
        for c in contours:
            x, y, w, h = cv2.boundingRect(c)
            if (w > 20 and h > 15) and w > 3*h and w<445 and h<40:
                cor  = get_cord(x,y,w,h)
                for i in cor:
                    if i == cord:
                        if drawing == True:
                            if mode == True:
                                cv2.rectangle(original_image,(x,y),(x+w,y+h),(0,0,255),2)
                        #print ("It's a box\n")
                        #print ('Dimension: '+ str((x+w)-x)+ " X "+ str((y+h)-y))

In [ ]:
cv2.namedWindow('camera',1)
cv2.setMouseCallback("camera",mouse_func)
while(1):
    cv2.imshow('camera', original_image)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    elif k == 27:
        break
cv2.destroyAllWindows()

# EXTRA
## Live Drawing

In [ ]:
drawing = False # true if mouse is pressed
mode = True # if True, draw rectangle. Press 'm' to toggle to curve
ix,iy = -1,-1

# mouse callback function
def draw_circle(event,x,y,flags,param):
    global ix,iy,drawing,mode

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            if mode == True:
                cv2.rectangle(img,(ix,iy),(x,y),(0,255,0),-1)
            else:
                cv2.circle(img,(x,y),5,(0,0,255),-1)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if mode == True:
            cv2.rectangle(img,(ix,iy),(x,y),(0,255,0),-1)
        else:
            cv2.circle(img,(x,y),5,(0,0,255),-1)

In [ ]:
img = np.zeros((512,512,3), np.uint8)
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

while(1):
    cv2.imshow('image',img)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    elif k == 27:
        break

cv2.destroyAllWindows()

In [19]:
image_1 = cv2.imread('test.jpg')
#Sharpening Image
image_2 = cv2.GaussianBlur(image_1,(0,0),3)
image_3 = cv2.addWeighted(image_1, 2.0, image_2, -0.9, 0)# Thresholding the image
#save sharped image
show_image(image_3)
cv2.imwrite("test_2.jpg",image_3)

True